In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install transformers datasets accelerate peft

In [6]:
import numpy as np
import evaluate

In [13]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import json
from peft import LoraConfig, get_peft_model, TaskType

# Carregar os dados
with open('/content/drive/MyDrive/tp3/base-sintetica.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Preparar o dataset
formatted_data = [{"input_text": f"Pergunta: {item['question']} Contexto: ", "target_text": f"Resposta: {item['answer']}"} for item in data]
dataset = Dataset.from_list(formatted_data)
train_test_split = dataset.train_test_split(test_size=0.3)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

# Inicializar o tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Função de pré-processamento
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["target_text"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Aplicar pré-processamento aos datasets de treinamento e validação
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [15]:
# Carregar o modelo GPT-2
model = GPT2LMHeadModel.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Configurar LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Correção: usar CAUSAL_LM em vez de SEQ_2_SEQ_LM
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["attn.c_attn", "attn.c_proj"]
)

model = get_peft_model(model, lora_config)

# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/tp3/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/tp3/logs",
)

# Criar o trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Treinar o modelo
trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,11.757418
2,No log,7.588318
3,No log,1.109222
4,No log,1.087124
5,No log,0.980742
6,No log,0.928745
7,No log,0.894033
8,4.363100,0.843231
9,4.363100,0.825398
10,4.363100,0.804769


TrainOutput(global_step=1300, training_loss=2.1434459158090444, metrics={'train_runtime': 336.3927, 'train_samples_per_second': 15.458, 'train_steps_per_second': 3.865, 'total_flos': 1365785129779200.0, 'train_loss': 2.1434459158090444, 'epoch': 20.0})

In [16]:
# Salvar o modelo treinado e o tokenizer
trainer.save_model("/content/drive/MyDrive/tp3/modelo")
tokenizer.save_pretrained("/content/drive/MyDrive/tp3/modelo")

('/content/drive/MyDrive/tp3/modelo/tokenizer_config.json',
 '/content/drive/MyDrive/tp3/modelo/special_tokens_map.json',
 '/content/drive/MyDrive/tp3/modelo/vocab.json',
 '/content/drive/MyDrive/tp3/modelo/merges.txt',
 '/content/drive/MyDrive/tp3/modelo/added_tokens.json')

In [23]:
import torch
import numpy as np
from torch.nn import functional as F

# Função para calcular a perplexidade manualmente
def calculate_perplexity(model, dataset, tokenizer):
    model.eval()

    # Preparar dados
    def collate_fn(batch):
        inputs = tokenizer([example['input_text'] for example in batch], return_tensors='pt', max_length=512, truncation=True, padding='max_length')
        labels = tokenizer([example['target_text'] for example in batch], return_tensors='pt', max_length=512, truncation=True, padding='max_length').input_ids
        inputs['labels'] = labels
        return inputs

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, collate_fn=collate_fn)

    # Mover modelo para a GPU se disponível
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    total_loss = 0
    total_tokens = 0

    for batch in dataloader:
        # Mover dados para o mesmo dispositivo que o modelo
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs.logits
            labels = batch['labels']

            # Shift logits and labels for calculating the loss
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id, reduction='sum')
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            total_loss += loss.item()
            total_tokens += shift_labels.numel() - (shift_labels == tokenizer.pad_token_id).sum().item()

    perplexity = np.exp(total_loss / total_tokens)
    return perplexity

# Calcular perplexidade no conjunto de validação
perplexity = calculate_perplexity(model, val_dataset, tokenizer)
print(f"Perplexity: {perplexity}")

Perplexity: 9661.036227619144


**Implementação de RAG**

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install sentence-transformers

In [66]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline as hf_pipeline

# Carregar o modelo de embeddings
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

# Função para extrair embeddings de texto
def get_embeddings(texts):
    embeddings = embedding_model.encode(texts, convert_to_tensor=True).cpu().numpy()
    return embeddings

# Carregar e indexar documentos
documents = []
document_path = "/content/drive/MyDrive/tp3/textos_extraidos"
for filename in os.listdir(document_path):
    if filename.endswith(".txt"):
        with open(os.path.join(document_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            documents.append(content)

# Verificar se documentos foram carregados corretamente
print(f"Total de documentos carregados: {len(documents)}")

# Extrair embeddings dos documentos
doc_embeddings = get_embeddings(documents)

# Verificar dimensões dos embeddings
print(f"Dimensões dos embeddings: {doc_embeddings.shape}")

# Criar o índice FAISS
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

# Carregar o modelo ajustado para geração de respostas
model_path = "/content/drive/MyDrive/tp3/modelo"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(model_path)

qa_pipeline = hf_pipeline("text-generation", model=model, tokenizer=tokenizer_gpt2)

# Função para busca de documentos
def search_documents(query, k=5):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, k)
    valid_indices = [i for i in indices[0] if i < len(documents)]
    results = [documents[i] for i in valid_indices]
    return results

# Função para gerar resposta
def generate_answer(question, retrieved_documents):
    context = "\n".join(retrieved_documents[:3])
    input_text = f"Resposta:"
    result = qa_pipeline(input_text, max_new_tokens=30, num_return_sequences=1)
    return result[0]['generated_text'].strip()

# Função para fazer uma pergunta
def ask_question(question):
    retrieved_docs = search_documents(question, k=5)
    if not retrieved_docs:
        return "Desculpe, não consegui encontrar informações relevantes."
    answer = generate_answer(question, retrieved_docs)
    return answer

Total de documentos carregados: 58
Dimensões dos embeddings: (58, 384)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [67]:
# Exemplo de uso
questions = [
    "Qual é o objetivo principal da Lei nº 12.089/2009?",
    "Como deve ser realizada a matrícula em cursos a distância?",
    "Qual é a responsabilidade do Poder Público em relação à acessibilidade?",
    "Quais são os direitos dos estagiários?"
]

for question in questions:
    try:
        answer = ask_question(question)
        print(f"{question}")
        print(f"{answer}")
        print("="*50)
    except IndexError as e:
        print(f"Erro ao processar a pergunta: {question}")
        print(e)
        print("="*50)

Qual é o objetivo principal da Lei nº 12.089/2009?
Resposta: 8 
2 - o que é a história. os do, e se, um. da forma do documento, forma- se um um.
Como deve ser realizada a matrícula em cursos a distância?
Resposta:= > "Pate para os resultados das de as atividades do da pesquisa poderá do aluno o trabalho que a seguir, a pontuação de
Qual é a responsabilidade do Poder Público em relação à acessibilidade?
Resposta: P e o

Um de seus exemplos de desenvolvimento para o C meio em que o documento poderá fazer o dolo poderá aprender de maneira e do
Quais são os direitos dos estagiários?
Resposta:

Alfredo, o “Homem do Mar do País de estudo das ciências e do do Governo para pós, pode aprender como o seu@


In [68]:
# Mais exemplos
questions = [
    "Qual é a política de matrícula tardia?",
    "Como funciona o trancamento de curso?",
    "Quais são as normas para transferência de curso?",
    "O que acontece se eu perder o prazo de matrícula?",
    "Quais são as regras para troca de turno?"
]

for question in questions:
    answer = ask_question(question)
    print(f"Pergunta: {question}")
    print(f"{answer}")
    print("="*50)

Pergunta: Qual é a política de matrícula tardia?
Resposta: "
, é feita de duas, do que todas as
contas - se os - forem também - ou é como, aprender
Pergunta: Como funciona o trancamento de curso?
Resposta: 1

Com base em dados, do clima adaptado, dados, é permitido o acesso a pessoas. na instituição. com links de
Pergunta: Quais são as normas para transferência de curso?
Resposta: A e o de a outros o é que as informações estão disponíveis que você poderá do do do no do dos outros forma para de informações na forma de
Pergunta: O que acontece se eu perder o prazo de matrícula?
Resposta: Seres a sua morte a seus amigos de ensino, as suas opções para os filhos, se for ainda a pessoa-guia de quem que estiver à
Pergunta: Quais são as regras para troca de turno?
Resposta: "Lista de itens de suporte do no no no poderá incluir no no programa.

Lista de programas ou artigos de
 é do
